<div style="text-align: center">
<b><font size=6>ChEn 3603 Homework 4 Problem 4
    </font></b>
</div>

<div class="alert alert-block alert-danger">

&copy; 2020 This material is copyright protected. Distributing this material in any form without written permission from Professor Sutherland is prohibited and may result in academic discipline including dismissal from the chemical engineering program and potentially from the university.

# Preliminaries

We are interested in calculating the time for a beaker of benzene to evaporate completely.
The following picture illustrates the coordinate system we will adopt.

![beaker diagram](beaker.png)

Note that the initial height of the liquid in the beaker is $h=H-L$ or $z=L$.

From the lecture notes, we know that
\begin{equation}
    N_A = \frac{c D_{AB}}{z-z_0}\ln\left(\frac{1-x_A}{1-x_A^0} \right)  
    \label{eq:Na}
\end{equation}

In [1]:
import numpy as np

def antoine( a, T ):
    #
    # Ps = antoine( a, T )
    #
    # Uses Antoine's equation to obtain the vapor pressure of a substance given
    # the coefficients of the equation:
    #  Ps = a1 - a2/(a3+T)
    #
    # INPUTS:
    #  a - the antoine coefficients with coeffients in columns and species in
    #      rows. Typically with T in Celsius and P in mmHg.
    #  T - the temperature to evaluate the vapor pressure at.  Typically in C
    #
    # OUTPUT:
    #  Ps - row vector of species vapor pressures at the specified temperature,
    #       typically in mmHg.
    #
    # The units depend on the units used for the coefficients.  The user is
    # responsible for maintaining consistency with units.  Customarily,
    # coefficients are supplied for pressure in mmHg and T in Celsius.

    Ps = 10.0**( a[:,0]-a[:,1] / ( a[:,2] + T ) )
    return Ps.squeeze()  # if a is a vector rather than a matrix, return a scalar

# Part 1

We begin with the mole balance for species $A$ in the _liquid phase_:
$$
    \d{}{t}\int_\mathsf{V}{c_{i}}=-\int_\mathsf{S}{\mathbf{N}_{i}}+\int_\mathsf{V}{S_{i}},
$$
we simplify due to no reaction and rewrite the fluxes as a total species flux rather than diffusive and convective components to find 
\begin{equation}
    \d{}{t}\int_\mathsf{V}{c_{A}^{\ell}}=-\int_\mathsf{S}{N_{A}}
    \label{eq:liquid-balance}
\end{equation}
Here, the $\ell$ superscript reminds us that we are describing the _liquid_ phase concentration.

A few observations:
 * $V=hA_{c}$ where $h=H-z$ is the liquid height and the cross-sectional area, $A_{c}$, is constant. 

 * The outward-pointed unit-normal is in the $-z$ direction since $+z$ is downward (consistent with SHR example 3.2), thus $N_{A}\cdot\mathbf{a}=-N_{A}$. 

 * The only flux is across the gas-liquid surface. 

 * The concentration in the _liquid_, $c_{A}^{\ell}$, is constant in space and time.

Therefore, \eqref{eq:liquid-balance} becomes
\begin{align}
    c_{A}^{\ell}A_{c}\d{h}{t}	&=	N_{A}A_{c}, \nonumber \\
    \d{h}{t}	&=	\frac{N_{A}}{c_{A}^{\ell}}. \label{eq:dhdt} 
\end{align}



In [2]:
T = 25+273.15  # temperature, K
P = 101325     # pressure, Pa
L = 0.005      # diffusion path length, m
H = 0.06       # initial liquid height, m

xa  = 1.0      # benzene mole fraction in liquid
ya0 = 0.0      # benzene mole fraction at top of beaker (z=0)

caL = 0.874*100**3 / (12*6+6) # molar concentration for liquid benzene

R = 8.315      # gas constant, (Pa m^3)/(mol K)
c = P/(R*T)    # molar concentration

Dab = 0.0905 * 100**-2  # diffusivity, m^2/s

# Benzene antoine equation coefficients
coefs = np.array([[ 4.01814, 1203.835, -53.226 ]])

Benzene Antoine equation parameters were obtained from the [NIST webbook](https://webbook.nist.gov/cgi/cbook.cgi?ID=C71432&Units=SI&Mask=4#Thermo-Phase)

We need to get a value for $N_A$.
To do that, we need the vapor mole fraction of benzene.
We use phase equilibrium conditions to get the Benzene mole fraction at the vapor side of the liquid interface:

In [3]:
Pa = antoine( coefs, T )  # T in K and P in bar
Pa *= 1e5  # convert from bar back to Pa

# K-value from Raoult's law:
Ka = Pa / P

yaL = xa * Ka  # benzene mole fraction in vapor at z=L.  We are calling this x_A^L in the equations

We can get a rough estimate of the time to evaporate all of the fluid by assuming $N_{A}$ is constant in time in \eqref{eq:dhdt}. 
Integrating this gives
\begin{align}
    \int_{H-L}^{0} dh &= \frac{N_A}{c_A^\ell} \int_{0}^t dt \nonumber \\
    L-H	&=	\frac{ N_{A} }{ c_{A}^{\ell} } t \nonumber \\
    t	&=	\frac{ c_{A}^{\ell} \left( L - H \right)}{ N_{A} } \label{eq:t-rough}
\end{align}
where $N_A$ is found from \eqref{eq:Na}.

In [4]:
# determine Na
Na = c*Dab / L * np.log( (1 - yaL) / (1 - ya0) )
t = caL * (L-H) / Na
# print(Na)
print('Rough estimate for time to go dry: {:.1f} hours'.format(t/3600))

Rough estimate for time to go dry: 17.3 hours


# Part 2

From \eqref{eq:Na}, we know that $N_{A}$ is a function of $z$ (and therefore of $h$).
We need to express this dependence before we can integrate \eqref{eq:dhdt}.

We can change variables between $h$ and $z$ by differentiating the relationship between them:
$$
    h = H-z \quad \implies \quad dh = -dz.
$$
This should match intuition since as $h$ increases, $z$ decreases (given our coordinate choice for $z$).
Making this change of variables, \eqref{eq:dhdt} becomes
\begin{equation}
    \d{z}{t} = -\frac{N_{A}}{c_{a}^{\ell}} \label{eq:dzdt}
\end{equation}
Now substituting \eqref{eq:Na}, we have
$$
    \d{z}{t} = -\frac{c D_{AB}}{c_{A}^{\ell} z} \ln\left( 1 - x_{A}^{L} \right).
$$
Here $c$ is the vapor molar concentration while $c^{\ell}$ is the liquid phase molar concentration. 
This can be separated and integrated as follows:
\begin{align}
    \int_{L}^{H}z\,dz	&=	-\frac{cD_{AB}}{c_{A}^{\ell}}\ln\left(1-x_{A}^{L}\right)\int_{0}^{t}dt, \\
    \frac{H^{2}-L^{2}}{2}	&=	-\frac{cD_{AB}}{c_{A}^{\ell}}\ln\left(1-x_{A}^{L}\right)t \\
    t	&=	\boxed{ \frac{ c_{A}^{\ell} \left( L^{2} - H^{2} \right) }{ 2 c D_{AB} \ln\left( 1 - x_{A}^{L} \right)} } \label{eq:t-rigorous}
\end{align}

In [5]:
tdry = caL * (L**2 - H**2) / ( 2*c*Dab * np.log(1-yaL) )
print('Our more rigorous estimate of the time to go dry: {:.1f} hours'.format(tdry/3600))

Our more rigorous estimate of the time to go dry: 112.5 hours


## Alternative approach for Part 2

An alternative way of solving this is to start with \eqref{eq:dhdt} and substitute out for $z=H-h$ in \eqref{eq:Na} to find 
$$
    \d{h}{t} = \frac{N_{A}}{c_{A}^{\ell}} = \frac{cD_{AB}}{c_{A}^{\ell}\left(H-h\right)}\ln\left(1-x_{A}^{L}\right) 
$$
which we may separated and integrated to find 
\begin{align}
    \int_{H-L}^{0}\left(H-h\right)dh	&=	\frac{cD_{AB}}{c_{A}^{\ell}}\ln\left(1-x_{A}^{L}\right)\int_{0}^{t}dt, \\
    -H\left(H-L\right)+\frac{\left(H-L\right)^{2}}{2}	&=	\frac{cD_{AB}}{c_{A}^{\ell}}\ln\left(1-x_{A}^{L}\right)t \\
    \frac{L^{2}-H^{2}}{2}	&=	\frac{cD_{AB}}{c_{A}^{\ell}}\ln\left(1-x_{A}^{L}\right)t \\
    t	&=	\frac{c_{A}^{\ell}\left(L^{2}-H^{2}\right)}{2cD_{AB}\ln\left(1-x_{A}^{L}\right)},
\end{align}
which is the same result as before.
Notice that the bounds on the integral change because of the different frame of reference for $h$ than for $z$.

## Discussion

The dramatic difference in these answers is due to the increase in the diffusion path length as the beaker empties, which decreases $N_{A}$ as can be seen in \eqref{eq:Na}. 
The effect of this is to _significantly_ increase the time that is required for the liquid to fully vaporize. 